In [25]:
import os
import time
import json
from typing import List, Dict, Any, Optional, Literal
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, Parallel, END
from langgraph.prebuilt import create_react_agent


ImportError: cannot import name 'Parallel' from 'langgraph.graph' (/home/pavan/Ds/pro/doctor-appointment-multiagent/venv/lib/python3.10/site-packages/langgraph/graph/__init__.py)